In [3]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 4.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

Wczytywanie danych

In [9]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [10]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [11]:
df.columns

Index(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       ...
       'param_pearl', 'param_stan', 'param_wersja', 'param_emisja-co2',
       'param_body-type', 'param_matowy', 'param_bezwypadkowy',
       'param_akryl-(niemetalizowany)', 'param_monthly-payment-value',
       'car_id'],
      dtype='object', length=155)

Sprawdzanie wszystkich kolumn nie jest optymalne

In [12]:
#dummy model
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [14]:
X = df[ ['car_id']].values
y = df['price_value'].values
model = DummyRegressor()
model.fit(X, y)
y_pred = model.predict(X)
mae(y, y_pred) #Benchmark - wartość średnia

39465.934630440985

In [15]:
#które kolumny zawierają słowo price
[x for x in df.columns if 'price' in x]

['price_currency', 'price_details', 'price_value']

In [16]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [18]:
#usuwamy wszystkie ogłoszenia z euro
df = df[ df['price_currency'] != 'EUR']
df.shape

(106290, 155)

Features - które cechy zasługują na naszą uwagę

In [0]:
SUFFIX_CAT = '__cat'
for feat in df.columns: #faktoryzacja cech
  if isinstance(df[feat][0], list): continue

  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [25]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)

151

In [26]:
X = df[cat_feats].values
y = df['price_value'].values
model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
np.mean(scores)

-19566.588937368324

In [28]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)
imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat


In [30]:
df['param_faktura-vat__cat'].value_counts

<bound method IndexOpsMixin.value_counts of 0        -1
2         0
3         0
4        -1
5         0
         ..
160609    0
160610    0
160611   -1
160614   -1
160615   -1
Name: param_faktura-vat__cat, Length: 106290, dtype: int64>

In [31]:
[x for x in df.columns if 'napęd' in x]

['param_napęd', 'param_napęd__cat']

In [32]:
df['param_napęd'].value_counts

<bound method IndexOpsMixin.value_counts of 0                                  None
2                      Na przednie koła
3         4x4 (dołączany automatycznie)
4                      Na przednie koła
5         4x4 (dołączany automatycznie)
                      ...              
160609                 Na przednie koła
160610                 Na przednie koła
160611                 Na przednie koła
160614    4x4 (dołączany automatycznie)
160615                 Na przednie koła
Name: param_napęd, Length: 106290, dtype: object>

In [33]:
df['param_napęd'].describe()

count                89947
unique                   5
top       Na przednie koła
freq                 65926
Name: param_napęd, dtype: object

In [34]:
df['param_napęd'].unique()

array([None, 'Na przednie koła', '4x4 (dołączany automatycznie)',
       'Na tylne koła', '4x4 (dołączany ręcznie)', '4x4 (stały)'],
      dtype=object)

In [36]:
df['param_napęd__cat'].unique()

array([-1,  0,  1,  2,  3,  4])

In [0]:
!git config --global user.email "interes@o2.pl"
!git config --global user.name "Rafal L"

In [0]:
!git add 'day3 '

In [38]:
ls

data/  day2_visualization.ipynb  README.md


In [39]:
pwd

'/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car'